In [24]:
import lyricsgenius as genius
api = genius.Genius("Uzv_tj6uOXE9_1eX2wCrOGi3j4JtmYcm1kVF-UmPcC5gJh1MQZKJLph4aFp9frFx")
artist = api.search_artist("Drake", max_songs=1)

Searching for songs by Drake...

Song 1: "0 to 100 / The Catch Up"

Reached user-specified song limit (1).
Done. Found 1 songs.


In [20]:
song = api.search_song('In My Feelings', 'Drake')

Searching for "In My Feelings" by Drake...
Done.


In [21]:
artist.add_song(song)

0

In [25]:
artist.save_lyrics(format_='txt')

AttributeError: 'Artist' object has no attribute 'artist'

In [1]:
import os,re
import requests as rq
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy

In [9]:
with open("data/rappers.txt", "w", encoding='utf-8') as f:
    f.write(str("\n".join(rappers)))

In [10]:
# make rappers from data folder
def get_rappers():
    
    def validate(name):
        # checks if the name has a
        illegal_chars = "|?:^*\""

        for c in illegal_chars:
            if c == '|':
                if name.find(c) > 0:
                    name = name.split("|")[1]
            if c == "\"":
                if name.find(c) > 0:
                    name = name.replace(r'"','',2)
            else: 
                name = name.replace(c,'')
        return name
    
    with open("data/rappers.txt", "r", encoding='utf-8') as f:
        rappers = f.read()
        
    return [validate(rapper).strip() for rapper in rappers.split("\n")]
rappers = get_rappers() 

In [235]:
import spotipy.oauth2 as oauth2

# When an error occurs saying the token expired, re-run this cell.
# Tokens are 1 hour I think.
client_id = 'e3344de9f0454df280cc34cdba040bf7'
client_secret = 'da09e980b2fb4f11a68a3c00a2669a7d'

credentials = oauth2.SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

In [218]:
get_uri("D'Angelo", 'artist')

D'Angelo


'spotify:artist:336vr2M3Va0FjyvB55lJEd'

In [229]:
def get_uri(name, q_type):
    result = spotify.search(
        q=r"%s" % name, 
        type=q_type)[q_type+"s"]['items']

    for x in result:
        if x['name'].lower() == name.lower():
            return x['uri']



In [238]:
with open("data/albums/Frank Ocean.txt", 'r') as f:
    x = eval(f.read())

In [245]:
x[0]['tracks'][1]

{'track_title': 'Ivy',
 'uri': 'spotify:track:2ZWlPOoWh0626oTaHrnl2a',
 'collaborators': ['Frank Ocean']}

In [230]:
def get_albums(artist):
    artist_id = get_uri(artist, 'artist')

    albums = spotify.artist_albums(artist_id, album_type='album')
    
    return set([album['name'] for album in albums['items']])

In [231]:
def get_album_info(album):
    info = {}
    
    album_id = get_uri(album,'album')
    try:
        album_info = spotify.album(album_id)
    except:
        raise Exception("Did not find album")
    try:
        song_info = spotify.album_tracks(album_id)
    except:
        raise Exception("Did not find album tracks")
    
    info['name'] = album_info['name']
    info['uri'] = album_info['uri']
    info['release_date'] = album_info['release_date']
    
    info['tracks'] = [{
        'track_title':track['name'],
        'uri':track['uri'],
        'collaborators':[artist['name'] for artist in track['artists']]}
        for track in song_info['items']]
    
    return info
    

In [223]:
errors = []
for r in (rappers):
    with open('data/albums/%s.txt' % r, 'r') as f:
        if len(f.read()) < 2:
            errors.append(r)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 4018: character maps to <undefined>

In [187]:
for e in errors:
    with open('data/albums/%s.txt' % r), 'w') as f:
        f.write(str(get_discography(e)))

AttributeError: 'NoneType' object has no attribute 'split'

In [232]:
def get_discography(artist):
    
    albums = get_albums(artist)
    
    info = []
    
    for album in albums:
        try:
            info.append(get_album_info(album))
        except:
            continue
    return info

In [233]:
errors = []
for i,rapper in enumerate(rappers):
    with open("data/albums/%s.txt" % rapper, "w", encoding='utf-8') as f:
        print(rapper)
        try:
            print("======%d======" % i)
            f.write(str(get_discography(rapper)))
        except AttributeError:
            print("===ERROR %d===" % i)
            errors.append((rapper,"AttributeError"))
            continue
        except IOError:
            print("===ERROR %d===" % i)
            errors.append((rapper, "IOError"))
            continue


03 Greedo
======0======
100 Kila
======1======
100s
======2======
12 Gauge
======3======
2 Chainz
======4======
2 Pistols
======5======
2$ Fabo
======6======
21 Savage
======7======
2Mex
======8======
360
======9======
40 Glocc
======10======
50 Cent
======11======
6ix9ine
======12======
6lack
======13======
The 6th Letter
======14======
9th Wonder
======15======
A+
======16======
===ERROR 16===
A-Q
======17======
Ab-Soul
======18======
A Boogie wit da Hoodie
======19======
Abstract Rude
======20======
Ace Hood
======21======
Aceyalone
======22======
Action Bronson
======23======
Adam Saleh
======24======
Aesop Rock
======25======
Afrika Bambaataa
======26======
Afroman
======27======
Afu-Ra
======28======
Agallah
======29======
Ahmad Lewis
======30======
Ajs Nigrutin
======31======
AKA
======32======
Akala
======33======
Akinyele
======34======
Akir
======35======
Akon
======36======
The Alchemist
======37======
Alex Wiley
======38======
Ali Vegas
======39======
AMG
======40======
Ami

SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=&limit=10&offset=0&type=artist:
 No search query

In [26]:
def get_albums(artist):
    
    artist_id = get_uri(artist, 'artist')
    albums = spotify.artist_albums(artist_id, album_type='album')
    # For some reason duplicates some albums. Wrap as a set
    return set([album['name'] for album in albums['items']])

In [21]:
def get_album_artists(album):
    album_id = get_uri(album, 'album')
    tracks = spotify.album_tracks(album_id)
    
    return set([artist['name']  
            for track in tracks['items']
            for artist in track['artists']])

In [117]:
def get_collaborators(artist):
    
    albums = get_albums(artist)

    return {album:list(get_album_artists(album)) for album in albums}

In [22]:
def get_album_tracks(album):
    album_id = get_uri(album, 'album')
    
    tracks = spotify.album_tracks(album_id)
    
    return [track['name'] for track in tracks['items']]

In [51]:
def get_info(artist):
    path = r"data/albums/%s.txt" % artist
    
    with open(path, "r") as f:
        info = eval(f.read())
    
    return info

In [171]:
def get_artist_features(artist, features = ['energy','tempo','valence','loudness']):
    path = r"data/albums/%s.txt" % artist
    
    with open(path, "r", encoding="utf-8") as f:
        info = eval(f.read())
    
    albums_info = []

    # for each album in info
    for album in info:
        
        info = {}
        
        # get the track ids
        track_ids = [track['uri'] for track in album['tracks']]
        
        info['name'] = album['name']
        info['date'] = album['release_date']
        
        # get the features of the tracks 
        album_features = spotify.audio_features(track_ids)

        # Some tracks have no features so it returns None.
        info['features'] = [{feature:np.mean([track[feature] for track in album_features if track != None])} 
                                  for feature in features]
        
        albums_info.append(info)
    return albums_info

In [191]:
get_artist_features("D'Angelo")

SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [190]:
err = """Amir Obè
Andy Samberg
Anthony Parker
Anybody Killa
Bang Yong-guk
Bob Doe
Boss
Busy Bee Starski
Cam'ron
Cashis
Casper Nyovest
Chi Ali
Chill Rob G
Cordaro Stewart
Crooked I
Currensy
D'Angelo
D'banj
Da'unda'dogg
Dee Barnes
Diamond
Disco King Mario
Ed O.G.
Focus...
Freaky Tah
Ganksta N-I-P
Gary
Half a Mill
Haitian Jack
Hodgy Beats
Honey Cocaine
J.I.D
J.R. Writer
Jackie Hill-Perry
Johntá Austin
Joji Miller
Jorma Taccone
Junhyung
K'naan
Malcolm David Kelley
Khleo
Knoc-turn'al
Kool Herc
Kwamé
Lakey The Kid
Lauryn Hill
LE
Lil Flip
Lil Herb
Lil' Fizz
Lil' Keke
Lil' Ronnie
Lil' Wil
Los
Luther Campbell
Mac
Maejor Ali
Magic
Marčelo
MC Pressure
MC Ride
Melle Mel
Mike Will Made It
Money-B
Monoxide Child
Mr. Porter
NBA YoungBoy
Nature
Nitty Scott, MC
Noah 40 Shebib
P. Reign
Roger Troutman
Roxanne Shanté
Royce da 5'9
Scotty
Sean Combs
Smooth
Solzilla
Stoupe the Enemy of Mankind
Suga
Suge Knight
Syd Tha Kyd
T. Mills
Tone Lōc
Too Short
Trick-Trick
Tristan Wilds
Tupac Shakur
Ty Dolla Sign
Tyra Bolling
U$O
Willow Smith
Yoon Mi-rae
Young Jeezy"""
err = err.split("\n")
print(len(err))

94


In [237]:
for rapper in rappers:
    with open("data/features/%s.txt" % rapper, 'r', encoding='utf-8') as f:
        data = get_artist_features(rapper)
        if len(str(data)) < 5:
            print (rapper)


2$ Fabo


SyntaxError: unexpected EOF while parsing (<string>, line 0)